## Fake News Classifier using Bidirectional LSTM

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
df.shape

(20800, 5)

In [ ]:
df=df.dropna()

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
## get the independent features
x=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
## check whether the dataset is balanced or not
y.value_counts()

,count
label,
0,10361
1,7924


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [ ]:
## vocabulary size
voc_size=5000

In [ ]:
messages=x.copy()

In [ ]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[3934, 1742, 638, 1833, 3297, 80, 3499, 3728, 3661, 4684],
 [359, 1456, 723, 4718, 2270, 918, 1236],
 [1623, 666, 985, 338],
 [417, 2972, 4557, 4075, 1923, 880],
 [1508, 2270, 1755, 2608, 4583, 3860, 2270, 940, 1910, 3520],
 [3682,
  4140,
  1570,
  3127,
  2891,
  3536,
  4015,
  1288,
  2595,
  4310,
  2470,
  1800,
  1706,
  2108,
  1236],
 [4037, 4688, 3274, 1467, 4827, 2548, 2368, 4014, 4158, 1106, 1237],
 [812, 4715, 3447, 4136, 3294, 1616, 3536, 494, 4158, 1106, 1237],
 [1553, 4192, 3519, 4193, 3488, 2493, 2518, 2594, 3536, 2262],
 [2055, 2650, 2820, 2078, 1990, 789, 171, 1076],
 [1819, 758, 840, 2205, 3158, 4608, 827, 3247, 2417, 3389, 4117],
 [4075, 1003, 3297, 2493, 3536, 3294],
 [2024, 1983, 2054, 748, 2428, 4043, 4682, 4910, 468],
 [868, 409, 4356, 3778, 1992, 4295, 4206, 4158, 1106, 1237],
 [4504, 2323, 306, 3301, 2035, 4158, 1106, 1237],
 [650, 392, 3199, 1791, 2758, 2085, 3124, 3371, 3798, 3215],
 [3338, 619, 1456],
 [1495, 2085, 1266, 1261, 3536, 2090, 2405, 1236],
 [4

## Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 3728, 3661, 4684],
       [   0,    0,    0, ..., 2270,  918, 1236],
       [   0,    0,    0, ...,  666,  985,  338],
       ...,
       [   0,    0,    0, ..., 4158, 1106, 1237],
       [   0,    0,    0, ..., 1700, 1519, 4683],
       [   0,    0,    0, ...,  742, 1387, 4770]], dtype=int32)

In [ ]:
## Creating model

# Define embedding vector size
embedding_vector_features = 40

# Ensure the model is built properly by specifying input shape
model = Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dense(1, activation='sigmoid'))

# Build the model explicitly
model.build(input_shape=(None, sent_length))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ (None, 200)                 │         112,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 23s 46ms/step - accuracy: 0.8183 - loss: 0.3545 - val_accuracy: 0.9135 - val_loss: 0.1995
Epoch 2/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 19s 42ms/step - accuracy: 0.9527 - loss: 0.1277 - val_accuracy: 0.9167 - val_loss: 0.2096
Epoch 3/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 16s 42ms/step - accuracy: 0.9736 - loss: 0.0760 - val_accuracy: 0.9162 - val_loss: 0.2748
Epoch 4/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.9833 - loss: 0.0491 - val_accuracy: 0.9137 - val_loss: 0.2730
Epoch 5/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.9916 - loss: 0.0318 - val_accuracy: 0.9114 - val_loss: 0.3576
Epoch 6/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - accuracy: 0.9964 - loss: 0.0145 - val_accuracy: 0.9100 - val_loss: 0.4188
Epoch 7/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 19s 42ms/step - accuracy: 0.9940 - loss: 0.0187 - val_accuracy: 0.9109 - val_loss: 0.4337
Epoch 8/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.9974 - loss: 0.0117 - 

In [ ]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


In [ ]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3151,  268],
       [ 298, 2318]])

In [ ]:
print(accuracy_score(y_test,y_pred))

0.9062137531068766
